In [1]:
# vLLM itself
!pip install vllm

# Hugging Face transformers for model handling
!pip install transformers

# BitsAndBytes (for quantization support like 4-bit/8-bit)
!pip install bitsandbytes

# Accelerate (required for model device mapping)
!pip install accelerate
# vLLM itself
!pip install vllm

# Hugging Face transformers for model handling
!pip install transformers

# BitsAndBytes (for quantization support like 4-bit/8-bit)
!pip install bitsandbytes

# Accelerate (required for model device mapping)
!pip install accelerate

# For caching models and logging
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.4/436.4 MB 3.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.0/180.0 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 58.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 63.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.1/888.1 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 80.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 89.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 14.5 MB/s eta 0:00:0000:01

In [2]:
import torch

# Check GPU status
print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"Total Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
print(f"Available Memory: {(torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_allocated(0)) / 1e9:.2f} GB")

# Clear cache
torch.cuda.empty_cache()

GPU: Tesla T4
Total Memory: 15.83 GB
Available Memory: 15.83 GB


In [ ]:
import os
from huggingface_hub import login

# Load token from environment variable
hf_token = os.getenv("HF_TOKEN")
login(token=hf_token)

In [4]:
import os
os.environ['VLLM_USE_V1'] = '0'

import torch
import gc

torch.cuda.empty_cache()
gc.collect()

if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        with torch.cuda.device(i):
            torch.cuda.empty_cache()
            torch.cuda.ipc_collect()

import pandas as pd
from vllm import LLM, SamplingParams
from tqdm import tqdm

df = pd.read_csv('/kaggle/input/banglachq/train.csv')
df = df[['id', 'question']].copy()
print(f"Loaded {len(df)} rows")

model_name = "google/gemma-3-1b-it"
print(f"\nLoading {model_name}...")

llm = LLM(
    model=model_name,
    trust_remote_code=True,
    dtype="float32",  # 1B should work with float16 on T4
    gpu_memory_utilization=0.7,
    max_model_len=2048,
    max_num_seqs=24,
)

tokenizer = llm.get_tokenizer()
print("Model loaded!\n")

# keep prompt intact
def translate_to_sylheti(bangla_text):
    prompt = f"""You are a precise translation tool. Your only task is to translate the given Bangla text to Sylheti dialect using Bengali script.

    INSTRUCTIONS:
    - Translate the Bangla text below to Sylheti dialect
    - Use only Bengali script (not Latin script or IPA)
    - Return ONLY the translated text with no additional commentary, explanations, or notes
    - Do not include phrases like "Here is the translation:" or "The Sylheti translation is:"
    - Do not add any metadata, formatting, or extra information
    - If you cannot translate a specific word, keep it as is in the original form

    Here are some examples of Bangla to Sylheti translations:
    Bangla: আপনি কি নিয়মিত স্ক্যান করেন?
    Sylheti: আফনে কিতা রেগুলার স্ক্যান খরইননি?

    Bangla: কোথায় কোচিং করেন?
    Sylheti: কোনানো কোচিং খরইন?

    Bangla: ডাল সিদ্ধ হয়েছে?
    Sylheti: ডাইল সিদ্ধ অইছেনি?

    Bangla: এই যে মিয়া, কি নিয়ে যাও?
    Sylheti: ওবা, কিতা লইয়া যাইরায়?

    Bangla: আপনি কি নিয়মিত স্ক্যান করেন?
    Sylheti: আফনে কিতা রেগুলার স্ক্যান খরইননি?

    Bangla text to translate:
    {bangla_text}

    Sylheti translation:"""
    
    messages = [{"role": "user", "content": prompt}]
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

sampling_params = SamplingParams(
    temperature=0.3,
    top_p=0.9,
    max_tokens=512,
    stop=["Bangla:"]
)

BATCH_SIZE = 24
all_translations = []

for batch_start in tqdm(range(0, len(df), BATCH_SIZE), desc="Translating"):
    batch_end = min(batch_start + BATCH_SIZE, len(df))
    batch_prompts = [translate_to_sylheti(row['question']) for _, row in df.iloc[batch_start:batch_end].iterrows()]
    
    outputs = llm.generate(batch_prompts, sampling_params)
    all_translations.extend([o.outputs[0].text.strip() for o in outputs])
    
    del outputs, batch_prompts
    gc.collect()
    torch.cuda.empty_cache()

df['sylheti_translation'] = all_translations
df.to_csv('gemma_5_shot_sylheti.csv', index=False)
print("Done!")


2025-09-27 22:40:14.803551: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759012815.143268      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759012815.237740      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


INFO 09-27 22:40:32 [__init__.py:216] Automatically detected platform cuda.
Loaded 1880 rows

Loading google/gemma-3-1b-it...
INFO 09-27 22:40:34 [utils.py:328] non-default args: {'trust_remote_code': True, 'dtype': 'float32', 'max_model_len': 2048, 'gpu_memory_utilization': 0.7, 'max_num_seqs': 24, 'disable_log_stats': True, 'model': 'google/gemma-3-1b-it'}


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


config.json:   0%|          | 0.00/899 [00:00<?, ?B/s]

INFO 09-27 22:40:50 [__init__.py:742] Resolved architecture: Gemma3ForCausalLM


...

Translating:  99%|█████████▊| 78/79 [16:23<00:13, 13.34s/it]

Adding requests:   0%|          | 0/8 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/8 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Translating: 100%|██████████| 79/79 [16:29<00:00, 12.52s/it]

Done!
